### Question 1: 

In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession \
 .builder \
 .master("local[*]") \
 .appName("test") \
 .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/29 05:40:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

### Question 2: 

In [8]:
!zcat fhv_tripdata_2019-10.csv.gz | head -n 101 > head.csv


gzip: stdout: Broken pipe


In [9]:
!head -n 10 head.csv

dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
B00009,2019-10-01 00:23:00,2019-10-01 00:35:00,264,264,,B00009
B00013,2019-10-01 00:11:29,2019-10-01 00:13:22,264,264,,B00013
B00014,2019-10-01 00:11:43,2019-10-01 00:37:20,264,264,,B00014
B00014,2019-10-01 00:56:29,2019-10-01 00:57:47,264,264,,B00014
B00014,2019-10-01 00:23:09,2019-10-01 00:28:27,264,264,,B00014
B00021         ,2019-10-01 00:00:48,2019-10-01 00:07:12,129,129,,B00021         
B00021         ,2019-10-01 00:47:23,2019-10-01 00:53:25,57,57,,B00021         
B00021         ,2019-10-01 00:10:06,2019-10-01 00:19:50,173,173,,B00021         
B00021         ,2019-10-01 00:51:37,2019-10-01 01:06:14,226,226,,B00021         


In [12]:
df=spark \
 .read \
 .option("header","true") \
 .csv("fhv_tripdata_2019-10.csv.gz")

In [30]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropOff_datetime: string (nullable = true)
 |-- PUlocationID: string (nullable = true)
 |-- DOlocationID: string (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [38]:
from pyspark.sql.types import *

In [39]:
schema = StructType([
    StructField("dispatching_base_num", StringType(), True),
    StructField("pickup_datetime", TimestampType(), True),
    StructField("dropOff_datetime", TimestampType(), True),
    StructField("PUlocationID", IntegerType(), True),
    StructField("DOlocationID", IntegerType(), True),
    StructField("SR_Flag", StringType(), True),
    StructField("Affiliated_base_number", StringType(), True)
])

In [40]:
df=spark \
 .read \
 .option("header","true") \
 .schema(schema) \
 .csv("fhv_tripdata_2019-10.csv.gz")

In [41]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [42]:
df = df.repartition(6)

In [44]:
df.write.parquet('fhv/2019/10/')

In [46]:
!ls

README.md  fhv	fhv_tripdata_2019-10.csv.gz  head.csv  homework.ipynb


In [47]:
!ls -lh fhv/2019/10/

total 39M
-rw-r--r-- 1 murat murat    0 Feb 29 06:14 _SUCCESS
-rw-r--r-- 1 murat murat 6.4M Feb 29 06:14 part-00000-0d4ffff5-1d3a-423b-9d60-e94b9680d917-c000.snappy.parquet
-rw-r--r-- 1 murat murat 6.4M Feb 29 06:14 part-00001-0d4ffff5-1d3a-423b-9d60-e94b9680d917-c000.snappy.parquet
-rw-r--r-- 1 murat murat 6.4M Feb 29 06:14 part-00002-0d4ffff5-1d3a-423b-9d60-e94b9680d917-c000.snappy.parquet
-rw-r--r-- 1 murat murat 6.4M Feb 29 06:14 part-00003-0d4ffff5-1d3a-423b-9d60-e94b9680d917-c000.snappy.parquet
-rw-r--r-- 1 murat murat 6.4M Feb 29 06:14 part-00004-0d4ffff5-1d3a-423b-9d60-e94b9680d917-c000.snappy.parquet
-rw-r--r-- 1 murat murat 6.4M Feb 29 06:14 part-00005-0d4ffff5-1d3a-423b-9d60-e94b9680d917-c000.snappy.parquet


### Question 3: 

In [49]:
from datetime import datetime

In [59]:
start=datetime(year=2019,month=10,day=15)

In [58]:
df.filter(F.to_date(df.pickup_datetime)==start).count()

62610

### Question 4:

In [64]:
from pyspark.sql.functions import col, unix_timestamp, round

In [71]:
df = df.withColumn("time_difference_seconds", unix_timestamp(col("dropOff_datetime")) - unix_timestamp(col("pickup_datetime")))

In [73]:
df = df.withColumn("length_hours", round((col("time_difference_seconds") / 3600), 2).cast("double"))

In [74]:
df.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------------------+------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|time_difference_seconds|length_hours|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------------------+------------+
|              B02653|2019-10-08 13:53:21|2019-10-08 13:56:24|         264|         247|   null|                B02653|                    183|        0.05|
|              B00837|2019-10-12 08:20:51|2019-10-12 08:32:35|         264|         264|   null|                B00837|                    704|         0.2|
|              B01315|2019-10-05 15:13:04|2019-10-05 15:19:48|         264|          74|   null|                B01315|                    404|        0.11|
|              B00789|2019-10-23 00:56:36|2019-10-23 01:33

In [79]:
# Find the maximum trip duration
max_trip_duration = df.agg({"length_hours": "max"}).collect()[0][0]

print("Maximum trip duration:", "{:.2f}".format(max_trip_duration), "hours")

Maximum trip duration: 631152.50 hours


### Question 5: